In [1]:
# This makes a first contrast curve for Altair

# created 2018 Sept. 20 by E.S.

## SECTION TO INITIALIZE

In [1]:
# import stuff

import urllib
import numpy as np
import matplotlib.pyplot as plt

import PynPoint

from PynPoint import Pypeline
from PynPoint.IOmodules.Hdf5Reading import Hdf5ReadingModule
from PynPoint.IOmodules.FitsWriting import FitsWritingModule
from PynPoint.IOmodules.FitsReading import FitsReadingModule
from PynPoint.IOmodules.TextReading import ParangReadingModule
from PynPoint.ProcessingModules import PSFpreparationModule, \
                                       PcaPsfSubtractionModule, \
                                       ContrastCurveModule, \
                                       FluxAndPosition, \
                                    StackingAndSubsampling
from PynPoint.ProcessingModules.StackingAndSubsampling import DerotateAndStackModule
from PynPoint.ProcessingModules.FluxAndPosition import FakePlanetModule
#from PynPoint.Util import AnalysisTools

In [2]:
# define workspaces and initialize Pypeline

working_place = "./pynpoint_experimentation_altair/working_place/oct27_working/"
input_place = "./pynpoint_experimentation_altair/input_place/oct27_input/"
output_place = "./pynpoint_experimentation_altair/output_place/oct27_output/"

pipeline = Pypeline(working_place_in=working_place,
                    input_place_in=input_place,
                    output_place_in=output_place)

# now a *.ini file has been generated (this includes the PIXSCALE), if no pre-existing one was there

Initiating PynPoint... [DONE]


In [3]:
## IF THE *INI FILE WAS NEWLY GENERATED, EDIT THE *INI FILE TO SET
## PIXSCALE = 0.0107

## SECTION TO READ IN DATA

In [4]:
# read in science FITS files 
# (should have PARANG=0 in all headers, so as to keep the PSF in the same original orientation on the array; 
# we need to wait to correct for PARANG later)

read_science = FitsReadingModule(name_in="read_science",
                                 input_dir=None,
                                 image_tag="science",
                                 check=True)

pipeline.add_module(read_science)

In [5]:
# confirm PIXSCALE change

pixscale_config = pipeline.get_attribute("config", "PIXSCALE")
print("Plate scale for /config/ is "+str(pixscale_config))
#pixscale_sci = pipeline.get_attribute("science", "PIXSCALE")
#print("Plate scale for /science/ is "+str(pixscale_sci))

Plate scale for /config/ is 0.0107


In [6]:
# read in PSF reference FITS files (i.e., unsaturated frames)
# (these do not have any PARANG in the header, but it probably doesn't matter)

read_ref_psf = FitsReadingModule(name_in="read_ref_psf",
                                 input_dir=input_place+'ref_psf/',
                                 image_tag="ref_psf",
                                 check=True)

pipeline.add_module(read_ref_psf)

## SECTION TO PROCESS DATA

In [7]:
# if you want to assign parallactic angles from text file to science frames

'''
parang = ParangReadingModule("fake_parang_vals.csv",
                             name_in="parang",
                             input_dir=input_place,
                             data_tag="science",
                             overwrite=True)

pipeline.add_module(parang)
'''

'\nparang = ParangReadingModule("fake_parang_vals.csv",\n                             name_in="parang",\n                             input_dir=input_place,\n                             data_tag="science",\n                             overwrite=True)\n\npipeline.add_module(parang)\n'

In [8]:
## TESTS

'''
## PIPELINE CHANNEL TEST: INJECT FAKE PLANET AND SEE IF THE PLANET PSF ITSELF GETS ROTATED

## THERE MIGHT BE 2 WAYS OF DOING THIS: WITH FAKE_PLANET OR WITH FAKEPLANETMODULE 

# test 1: images all have set PA=0

fake_planet_injection = FakePlanetModule(position=(0.32,0),
                                         magnitude=0,
                                         psf_scaling=2,
                                         interpolation="spline",
                                         name_in="fake_planet_injection",
                                         image_in_tag="science",
                                         psf_in_tag="ref_psf",
                                         image_out_tag="test_planet_inject")

pipeline.add_module(fake_planet_injection)


## PIPELINE CHANNEL 1: IMAGE RESIDUALS

# derotate and median the science images based on physical 
# parallactic angles

derotate_sci = DerotateAndStackModule(name_in="derotate_sci",
                                        image_in_tag="test_planet_inject",
                                        image_out_tag="derotated_test_planet_inject",
                                        derotate=True,
                                        stack=None,
                                        extra_rot=0.0)

pipeline.add_module(derotate_sci)


# write out fake planet-injected images 
write_fake_injection = FitsWritingModule(file_name="fake_injection_tests.fits",
                              name_in="write_fake_injection",
                              output_dir=output_place,
                              data_tag="test_planet_inject")

pipeline.add_module(write_fake_injection)

# write out derotated, PSF-subtracted images
write_all_derotated_psf_subted = FitsWritingModule(file_name="derotated_fake_injection_tests.fits",
                                                   name_in="write_all_derotated_psf_subted",
                                                   output_dir=output_place,
                                                   data_tag="derotated_test_planet_inject")

pipeline.add_module(write_all_derotated_psf_subted)


pipeline.run()
'''

'\n## PIPELINE CHANNEL TEST: INJECT FAKE PLANET AND SEE IF THE PLANET PSF ITSELF GETS ROTATED\n\n## THERE MIGHT BE 2 WAYS OF DOING THIS: WITH FAKE_PLANET OR WITH FAKEPLANETMODULE \n\n# test 1: images all have set PA=0\n\nfake_planet_injection = FakePlanetModule(position=(0.32,0),\n                                         magnitude=0,\n                                         psf_scaling=2,\n                                         interpolation="spline",\n                                         name_in="fake_planet_injection",\n                                         image_in_tag="science",\n                                         psf_in_tag="ref_psf",\n                                         image_out_tag="test_planet_inject")\n\npipeline.add_module(fake_planet_injection)\n\n\n## PIPELINE CHANNEL 1: IMAGE RESIDUALS\n\n# derotate and median the science images based on physical \n# parallactic angles\n\nderotate_sci = DerotateAndStackModule(name_in="derotate_sci",\n                 

In [7]:
## PIPELINE CHANNEL 1: IMAGE RESIDUALS

# generate PCA basis from unsaturated frames and
# do PCA PSF subtraction of the saturated frames

pca_pca_subt = PcaPsfSubtractionModule(pca_numbers=(5, ),
                                       name_in="pca",
                                       images_in_tag="science",
                                       reference_in_tag="ref_psf",
                                       res_mean_tag="mean_residuals",
                                       res_median_tag="median_residuals",
                                       res_arr_out_tag="all_resids",
                                       res_rot_mean_clip_tag="resid_rot",
                                       basis_out_tag="pca_components",
                                       subtract_mean=True,
                                       verbose=True)

pipeline.add_module(pca_pca_subt)

# note:
# images_in_tag: science images
# reference_in_tag: reference images, which COULD be the science images

In [7]:
## PIPELINE CHANNEL 2: MAKE CONTRAST CURVE

# make a contrast curve 
# (N.b. this does not separately require PcaPsfSubtractionModule)

'''
cent_size: mask radius
'''

# N.b. scale the reference PSF by 3.28 to match Altair's amplitude

contrast_curve = ContrastCurveModule(name_in="contrast_curve",
                            image_in_tag="science",
                            psf_in_tag="ref_psf",
                            contrast_out_tag="contrast_landscape",
                            pca_out_tag="pca_resids",
                            pca_number=5,
                            psf_scaling=3.28,
                            separation=(0.35, 0.40, 0.05), 
                            angle=(90.0, 360.0, 360.0), 
                            magnitude=(5, 1.0),
                            cent_size=None)

pipeline.add_module(contrast_curve)

In [8]:
# write PCA resids

write_pca_resids = FitsWritingModule(file_name="junk_pca_residuals.fits",
                              name_in="write_pca_resids",
                              output_dir=output_place,
                              data_tag="pca_resids")

pipeline.add_module(write_pca_resids)

In [10]:
# write out PCA basis set

write_pca_comp = FitsWritingModule(file_name="junk_pca_components.fits",
                              name_in="write_pca_comp",
                              output_dir=output_place,
                              data_tag="pca_components")

pipeline.add_module(write_pca_comp)

In [11]:
# write out PCA-based PSF-subtracted residuals out

write_mean_resids = FitsWritingModule(file_name="junk_mean_residuals.fits",
                              name_in="write_mean_resids",
                              output_dir=output_place,
                              data_tag="mean_residuals")

pipeline.add_module(write_mean_resids)

In [9]:
# write out PCA-based PSF-subtracted residuals out

write_median_resids = FitsWritingModule(file_name="junk_median_residuals.fits",
                              name_in="write_median_resids",
                              output_dir=output_place,
                              data_tag="median_residuals")

pipeline.add_module(write_median_resids)

In [9]:
pipeline.run()

Validating Pypeline... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute DIT (=ESO DET DIT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LATITUDE (=ESO TEL GEOLAT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LONGITUDE (=ESO TEL GEOLON) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute EXP_NO (=ESO DET EXP NO) not found in the FITS header.
  "header." % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute ND

Running FitsReadingModule... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute INSTRUMENT (=INSTRUME) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute DATE (=DATE-OBS) not found in the FITS header.
  "header." % (item[0], fitskey))


Running FitsReadingModule... [DONE]
Running ContrastCurveModule...
Processing position 1 out of 2.

...

/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/ProcessingModules/DetectionLimits.py:327: UserWarning: Magnitude decreases but false positive fraction increases. Adjusting magnitude to 3.265625 and step size to 0.0078125
  "to %s" % (list_mag[-3], mag_step/2.))


....

/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/ProcessingModules/DetectionLimits.py:327: UserWarning: Magnitude decreases but false positive fraction increases. Adjusting magnitude to 3.2578125 and step size to 0.00390625
  "to %s" % (list_mag[-3], mag_step/2.))


..

KeyboardInterrupt: 

In [13]:
contrast_curve_results = pipeline.get_data("contrast_landscape")
print(contrast_curve_results)

[]


In [12]:
#####################################################################################################################

In [13]:
# make plots

f, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(contrast_curve_results[:,0],contrast_curve_results[:,1])
ax1.set_title('Azim. averaged contrast limit')
ax1.set_xlabel('Radius (asec)')
ax2.set_ylabel('del_mag')
ax2.plot(contrast_curve_results[:,0],contrast_curve_results[:,3])
ax2.set_title('Threshold of FPF')
ax2.set_xlabel('Radius (asec)')
f.savefig('test.png')